<a href="https://colab.research.google.com/github/Yashasvinimr/27_RainbowBrigade_HealthAndWellness/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

# Path to your dataset folder in Google Drive
dataset_path = '/content/drive/MyDrive/Hackathon/Healthy teeth'

# Check the contents of the folder
print(os.listdir(dataset_path))

['h25.jpg', 'h23.jpg', 'h20.jpg', 'h17.jpg', 'h18.jpg', 'h16.jpg', 'h15.jpg', 'h13.jpg', 'h11.jpg', 'h12.jpg', 'h14.jpg', 'h9.jpg', 'h10.jpg', 'h8.jpg', 'h7.jpg', 'h34.jpg', 'h39.jpg', 'h43.jpg', 'h42.jpeg', 'h41.jpg', 'h45.jpg', 'h44.jpg', 'h46.jpg', 'h40.jpg', 'h38.jpg', 'h47.jpg', 'h49.jpg', 'h51.jpg', 'h52.jpg', 'h53.jpg', 'h50.jpg', 'h48.jpg', 'h37.jpg', 'h54.jpg', 'h28.jpg', 'h31.jpeg', 'h29.jpg', 'h27.jpg', 'h22.jpg', 'h35.jpg', 'h30.jpg', 'h32.jpg', 'h33.jpg', 'h36.jpg', 'h26.jpg', 'h21.jpg', 'h24.jpg', 'h19.jpg', 'h1.jpg', 'h55.jpg', 'h4.jpg', 'h6.jpg', 'h5.jpg', 'h3.jpg', 'h2.jpg']


In [10]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img

# Folder to save augmented images
augmented_path = '/content/drive/MyDrive/Hackathon/augmented_images_healthy'
os.makedirs(augmented_path, exist_ok=True)

# Augmentation settings
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Iterate through all images in the dataset folder
i = 1
for file_name in os.listdir(dataset_path):
    if file_name.lower().endswith(('.png', '.jpeg', '.jpg')):  # Check valid image types
        file_path = os.path.join(dataset_path, file_name)

        # Load and prepare the image
        img = load_img(file_path)        # Load image
        img_array = img_to_array(img)   # Convert to numpy array
        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape for generator

        # Augment and save images
        for batch in datagen.flow(img_array, batch_size=1):
            # Ensure the directory exists before saving the image
            save_path = os.path.join(augmented_path, f'{i}.jpg')
            os.makedirs(os.path.dirname(save_path), exist_ok=True)

            save_img(save_path, batch[0])  # Save as 1.jpg, 2.jpg, ...
            i += 1
            if i > 100:  # Generate a maximum of 100 augmented images
                break

In [13]:
# Define paths
import albumentations as A
import cv2
input_dir = '/content/drive/MyDrive/Hackathon/Healthy teeth'  # Replace with your folder containing images
output_dir = '/content/drive/MyDrive/Hackathon/healthy_gum_aug'  # Folder to save augmented images

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define augmentation pipeline using Albumentations
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.5),
])

# Get list of images
image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Initialize counter for renaming
counter = 1

# Process each image
for image_file in image_files:
    # Load image
    img_path = os.path.join(input_dir, image_file)
    image = cv2.imread(img_path)

    if image is None:
        print(f"Could not load image {image_file}. Skipping.")
        continue

    # Perform augmentation
    augmented = augment(image=image)
    augmented_image = augmented['image']

    # Save augmented image with sequential name
    output_path = os.path.join(output_dir, f"{counter}.jpg")
    cv2.imwrite(output_path, augmented_image)
    counter += 1

print(f"Augmented images saved to {output_dir}")


Augmented images saved to /content/drive/MyDrive/Hackathon/healthy_gum_aug


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to the dataset
dataset_path = '/content/drive/MyDrive/Hackathon'  # Update this path

# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values between 0 and 1
    validation_split=0.2     # Split dataset into training (80%) and validation (20%)
)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,
    class_mode='categorical',  # Multiclass classification
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 5549 images belonging to 4 classes.
Found 1384 images belonging to 4 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 output classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stopping]
)

# Save the trained model
model.save('/content/drive/MyDrive/Hackathon/tooth_classifier_model.keras')


Epoch 1/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 40s 208ms/step - accuracy: 0.6105 - loss: 0.9620 - val_accuracy: 0.7731 - val_loss: 0.5663
Epoch 2/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 38s 200ms/step - accuracy: 0.7979 - loss: 0.5147 - val_accuracy: 0.7796 - val_loss: 0.5920
Epoch 3/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 35s 167ms/step - accuracy: 0.8380 - loss: 0.4209 - val_accuracy: 0.8410 - val_loss: 0.4564
Epoch 4/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 30s 167ms/step - accuracy: 0.8673 - loss: 0.3552 - val_accuracy: 0.8056 - val_loss: 0.5128
Epoch 5/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 41s 167ms/step - accuracy: 0.8880 - loss: 0.2993 - val_accuracy: 0.8750 - val_loss: 0.3612
Epoch 6/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 40s 164ms/step - accuracy: 0.9216 - loss: 0.2306 - val_accuracy: 0.8750 - val_loss: 0.3309
Epoch 7/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 41s 166ms/step - accuracy: 0.9409 - loss: 0.1728 - val_accuracy: 0.8801 - val_loss: 0.3860
Epoch 8/20
174/174 ━━━━━━━━━━━━━━━━━━━━ 29s 164ms/step - accuracy: 0.9518 - loss: 0

In [7]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from google.colab import files

# Load the trained model
model = load_model('/content/drive/MyDrive/Hackathon/tooth_classifier_model.keras')

# Class labels corresponding to folder names
class_names = ['Gingivitis', 'Healthy_Gum_Aug', 'Mouth_Ulcer_Augmented_DataSet', 'Tooth_Discoloration_Augmented_DataSet']

# Function to predict the uploaded image
def predict_image(image):
    img = load_img(image, target_size=(150, 150))  # Resize image to match model input
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Predict the class of the image
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]  # Get class name with highest probability

    return predicted_class

# Upload image from local system
uploaded = files.upload()  # Upload the file manually
uploaded_image_path = list(uploaded.keys())[0]  # Get the uploaded file name

# Predict for the uploaded image
result = predict_image(uploaded_image_path)
print(f"The uploaded image is classified as: {result}")


Saving gingi.jpg to gingi.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
The uploaded image is classified as: Gingivitis
